In [37]:
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator, date2num
from IPython.display import HTML

d = "Data//"
flis = ["LT1", "LT2", "LT3", "LT4", "LT5", "LT6", "LT7", "LT8"]
OB = ["16 Hz","31.5 Hz","63 Hz","125 Hz","250 Hz","500 Hz","1 kHz","2 kHz","4 kHz","8 kHz","16 kHz"]
sheet = ["Leq","L90(LN4)"]
weather = pd.read_excel(d+"Weather.xlsx")
weather['Time'] = pd.to_datetime(weather['Time'].astype(str))
wdir = ["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW","---"]
lt = [[[] for i in range(2)] for j in range(8)]
dca, dcb = '07:00', '04:00'
lcrit = 1

for i, j in enumerate(flis):
    for k,l in enumerate(sheet):
        lt[i][k] = pd.read_excel(d+flis[i]+".xlsx", sheet_name=l)
        lt[i][k]['Start Time'] = pd.to_datetime(lt[i][k]['Start Time'].astype(str))
        lt[i][k] = pd.merge_asof(lt[i][k],weather,left_on="Start Time", right_on="Time")
        lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
        lt[i][k] = lt[i][k].set_index('Start Time')
        lt[i][k] = lt[i][k].tail(lt[i][k].shape[0] - 24) # removing first 6 hours from tail
        lt[i][k] = lt[i][k].head(lt[i][k].shape[0] - 24) # removing first 6 hours from head

%matplotlib widget
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, sharex=ax1)

def pr1(n, lw, c, w):
    n = int(n-1)
    if lw == 'Leq':
        lwv = 0
    else:
        lwv = 1
    ltall = lt[n][lwv]
    start = ltall.index.min()
    end = start+pd.Timedelta(hours=24)
    ltall = ltall[start:end] # between two datetimes
    ltall = ltall[ltall["Rain"]<0.2]
    ltall = ltall[ltall["Speed"]<=5]
    ltall = ltall[ltall["Dir"].isin(w)]
    if c:
        ltdall = ltall.between_time(dca, dcb) #filtering for dawn chorus
    return ltall, start, end

def table(n, lw, c, w):
    ltall, start, end = pr1(n, lw, c, w)
    ltall_av,ltall_min,ltall_max = ltall['Main'].mean(),ltall['Main'].min(),ltall['Main'].max()
    df = {'Date / Time':[start],'Arithmetic Mean (dBA)':ltall_av,'Min (dBA)':ltall_min,'Max (dBA)':ltall_max}
    table1 = pd.DataFrame(df).round(1)
    display(HTML(table1.to_html(index=False,notebook=True)))

def plot(n, lw, c, w):
    ax1.clear(), ax2.clear()
    ltall, start, end = pr1(n, lw, c, w)
    bins = range(15,80)
    if lw == 'Leq':
        ax1.set_ylabel('L$_{EQ}$ dBA')
    else:
        ax1.set_ylabel('L$_{90}$ dBA')
    ax1.scatter(ltall.index,ltall['Main'],c='black',marker='.')
    ltall_OB = ltall[OB].T
    start, end = date2num(start), date2num(end)
    ax2.imshow(ltall_OB, aspect="auto", origin='lower', extent=[start,end,0,len(OB)])
    myFmt = DateFormatter('%H:%M')
    minutes = MinuteLocator(interval = 240)
    ax1.xaxis.set_major_locator(minutes)
    ax1.xaxis.set_major_formatter(myFmt)
    tickloc = [x + 0.5 for x in range(0, len(OB))]
    ax2.yaxis.set(ticks=tickloc, ticklabels=OB)
    ax2.set_yticklabels(OB)
    ax1.set_xlabel('Time')
    ax2.set_xlabel('Time')
    fig.autofmt_xdate()
    fig.show()

box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='stretch')
box_layout2 = widgets.Layout(display='inline-flex',flex_flow='row',align_items='stretch',margin='0px 0px 10px 45px')
aw = widgets.Select(options=list(range(1,9)),description='LT: ',disabled=False)
lw = widgets.ToggleButtons(options=['Leq','L90'],description='Stat. level: ',disabled=False, layout=dict(width='250px'))
#bw = widgets.ToggleButtons(options=['Day','Night'],description='Period: ',disabled=False, layout=dict(width='250px'))
cw = widgets.Checkbox(value=False,description='Filter dawn Chorus',disabled=False)
wd = widgets.SelectMultiple(options=wdir,value=wdir,description='wind dir', disabled=False)
out = widgets.interactive_output(plot, {'n': aw, 'lw': lw, 'c': cw,'w':wd})
outt = widgets.interactive_output(table, {'n': aw, 'lw': lw, 'c': cw,'w':wd})
ui = widgets.VBox([widgets.HBox([lw, cw],layout=box_layout2),widgets.HBox([aw, wd])],layout=box_layout)
ui2 = widgets.HBox([outt],layout=widgets.Layout(margin='0px 0px 0px 90px'))
display(ui2,ui,out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()